In [1]:
## imports 
import pandas as pd
import numpy as np
import datetime
import math

## The data

The dataset is gathered from [Boulder open data](https://open-data.bouldercolorado.gov/datasets/053aadbad62c4ef784d1fbece60adedd_0/data) which is in Colorado, USA. It contains differenct EV charges in the area, with location, start time, charging time, energy (kWh) and more. Spans from 1/1/18 - 31/12/2020. 

In [2]:
df = pd.read_csv("../data/Electric_Vehicle_Charging_Station_Energy_Consumption.csv")
df.head()

,Station_Name,Station_Address,City,State_Province,Postal_Code,Transaction_Date,Transaction_Start_Time,Charging_Time__minutes_,Energy__kWh_,GHG_Savings__kg_,Gasoline_Savings__gallons_,Port_Type,ObjectId
0,COMM VITALITY / 1104 SPRUCE1,1100 Spruce St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:43:00+00,128,2.84,1.19,0.36,Level 2,1
1,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:16:00+00,68,5.66,2.38,0.71,Level 2,2
2,BOULDER / REC CENTER ST2,1305-1335 Gillaspie Dr,Boulder,Colorado,80305,2018/01/31 00:00:00+00,2021/01/21 13:52:00+00,73,3.94,1.65,0.49,Level 2,3
3,COMM VITALITY / 1500PEARL1,1500 Pearl St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 14:47:00+00,-,NaN,NaN,NaN,Level 2,4
4,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 13:54:00+00,40,4.42,1.86,0.55,Level 2,5


In [3]:
df.shape

(22580, 13)

### Remove nan values

Remove nan values for the `Charging_Time__minutes_` and replace , with .

In [26]:
df = df[(df['Charging_Time__minutes_'] != '     -       ') &
        (df['Charging_Time__minutes_'] != '    -      ') &
        (df['Charging_Time__minutes_'] != '   -     ') &
        (df['Charging_Time__minutes_'] != '  -    ') &
        (df['Charging_Time__minutes_'] != ' -   ')] # Remove "nan" values

df['Charging_Time__minutes_'] = df['Charging_Time__minutes_'].str.replace(',','.', regex=True) # Replace , with . in the strings

print(df.shape)
print(22580-df.shape[0], "rows of data are lost.")

(20837, 13)
1743 rows of data are lost.


Remove the remaining nan values for `Enerygy__kWh`.

In [27]:
df = df[df['Energy__kWh_'].notna()]

### Convert column types

In [28]:
# convert to float64
df['Charging_Time__minutes_'] = df['Charging_Time__minutes_'].astype(np.float64)

### Remove values that does make sense

**Remove charging times**
- Charging times has to be bigger than 2 minutes and smaller than 24 hours.

I choose 2 minutes as if a car is charged for this little time we assume that the driver changed his/hers mind and the data is therefore not relevant.

In [29]:
print("Observations lost:", df[df['Charging_Time__minutes_'] <= 2].shape[0])
df = df[df['Charging_Time__minutes_'] > 2]

Observations lost: 249


The 24 hours limit is choosen as most sessions lasts a few hours and charging sessions longer than 24 hours could be due to a minipulation error from the customer.

In [30]:
df[df['Charging_Time__minutes_'] > 24*60]

,Station_Name,Station_Address,City,State_Province,Postal_Code,Transaction_Date,Transaction_Start_Time,Charging_Time__minutes_,Energy__kWh_,GHG_Savings__kg_,Gasoline_Savings__gallons_,Port_Type,ObjectId
22479,BOULDER / BOULDER PARK S2,1739 Broadway,Boulder,Colorado,80302,2020/12/22 00:00:00+00,2021/01/21 20:22:00+00,1532.18,85.2,35.784,10.693,Level 2,22480


In [31]:
print("Observations lost:", df[df['Charging_Time__minutes_'] > 24*60].shape[0])
df = df[df['Charging_Time__minutes_'] < 24*60]

Observations lost: 1


**Energy usage**
- Energy consumption has to be postive and less than 100 kWh.

The procdure for the charging times also ensures that we have no observations with negative or 0 energy usage.

In [32]:
print("Observations lost:", df[df['Energy__kWh_'] <= 0].shape[0])
df = df[df['Energy__kWh_'] > 0]

Observations lost: 0


Furthermore a the upper limit is choosen as it is similar to the highest capacity of the Tesla Model S which is the EV with the biggest battery amongst the moest widespread models (p. 33, *A review of electric vehicle load open data...*).

In [33]:
print("Observations lost:", df[df['Energy__kWh_'] >= 100].shape[0])
df = df[df['Energy__kWh_'] < 100]

Observations lost: 0


### Datetime transformation

In the dataframe the date of the transaction is saved under `Transaction_Date`, where the time is not specified. The excat start time of the transaction is specified under the `Transaction_Start_Time`. I assume that the charging station is idle when the `Charging_Time__minutes_`has passed after the `Transaction_Start_Time`. 

In [34]:
# convert start time to datetime
df['start_Time_Datetime'] = pd.to_datetime(df['Transaction_Start_Time'])
# convert transaction date to datetime
df['Transaction_Date_Datetime'] = pd.to_datetime(df['Transaction_Date'])
# reset index of dataframe:
df = df.reset_index(drop=True)

In [35]:
# combine time and date and add the charging time to form the start and end time
end_times = []
start_times = []
for i in range(0,len(df)):
    dt = df.loc[i]['Transaction_Date_Datetime'].date() # get the date
    tm = df.loc[i]['start_Time_Datetime'].time() # get the time
    new_datetime = datetime.datetime.combine(dt,tm) # combine date and time into new datetime object
    start_times.append(new_datetime) # append start time
    add_minutes = df.loc[i]['Charging_Time__minutes_'] # get charging minutes
    end_times.append(new_datetime + datetime.timedelta(minutes=add_minutes)) # add charging minutes to time)

df['start_Datetime'] = start_times
df['end_Datetime'] = end_times

## Remove observations from 2020
Due to the Covid-19 pandemic the amount of charging observations was drastically reduced in 2020. The idle times will also be influenced by this pandemic and the data for this year might not be usable as this is not considered "normal" behaviour.

In [36]:
df_covid = df
df_covid['year'] = df_covid.start_Datetime.dt.year
df_covid = df_covid[df_covid['year'] < 2020]
print("Observations lost:", df.shape[0]-df_covid.shape[0])
print("Charging stations lost:", len(df.Station_Name.unique()) - 
      len(df_covid.Station_Name.unique()))

Observations lost: 4502
Charging stations lost: 5


In [37]:
df = df_covid

## Idle time
The idle time for an observation, *i*, is defined as the time between that observations end time and the next observations, *i+1*, start time. The last recorded observation for each of the charging stations therefore has no value.

In [38]:
# create a dictionary which contains the idle time for every index of an observation
idle_dict = dict.fromkeys(df.index)

In [39]:
# calculate idle time and save in dictionary
last_obs = [] # list for the last observations
names = df.Station_Name.unique()
for name in names: 
    temp = temp = df[df['Station_Name'] == name] # Slice dateframe for charging station
    temp = temp.sort_values(by='start_Datetime') # sort by the start time
    ind = temp.index # get index of temporary dataframe
    for i in range(0,len(ind)):
        if i == len(ind)-1: # if last observation
            last_obs.append(ind[i])
        else:
            time_diff = temp.loc[ind[i+1]]['start_Datetime'] - temp.loc[ind[i]]['end_Datetime']
            idle_dict[ind[i]] = time_diff.total_seconds()/60 # Append idle

In [40]:
df['idle_time_minutes'] = idle_dict.values()
df['idle_time_minutes'] = df['idle_time_minutes'].astype(np.float64) # change to float

To perform analysis we have to remove the rows with no/nan idle time.

In [41]:
print("Rows removed:", df.isnull().sum().idle_time_minutes)
df = df[df['idle_time_minutes'].notna()]

Rows removed: 22


Some idletimes are however below 0.

In [42]:
print("Observations with negative idle time:", df[df['idle_time_minutes'] < 0].shape[0])

Observations with negative idle time: 2249


In [43]:
temp = df[df['Station_Name'] == 'COMM VITALITY / 1000WALNUT1'].sort_values(by='start_Datetime')
temp = temp.reset_index()
temp.loc[5:8]

,index,Station_Name,Station_Address,City,State_Province,Postal_Code,Transaction_Date,Transaction_Start_Time,Charging_Time__minutes_,Energy__kWh_,GHG_Savings__kg_,Gasoline_Savings__gallons_,Port_Type,ObjectId,start_Time_Datetime,Transaction_Date_Datetime,start_Datetime,end_Datetime,year,idle_time_minutes
5,27,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/02/02 00:00:00+00,2021/01/21 11:31:00+00,118.0,6.52,2.74,0.82,Level 2,29,2021-01-21 11:31:00+00:00,2018-02-02 00:00:00+00:00,2018-02-02 11:31:00,2018-02-02 13:29:00,2018,129.0
6,22,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/02/02 00:00:00+00,2021/01/21 15:38:00+00,133.0,14.06,5.91,1.77,Level 2,24,2021-01-21 15:38:00+00:00,2018-02-02 00:00:00+00:00,2018-02-02 15:38:00,2018-02-02 17:51:00,2018,-10.0
7,19,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/02/02 00:00:00+00,2021/01/21 17:41:00+00,155.0,14.66,6.16,1.84,Level 2,21,2021-01-21 17:41:00+00:00,2018-02-02 00:00:00+00:00,2018-02-02 17:41:00,2018-02-02 20:16:00,2018,-142.0
8,20,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/02/02 00:00:00+00,2021/01/21 17:54:00+00,117.0,8.63,3.63,1.08,Level 2,22,2021-01-21 17:54:00+00:00,2018-02-02 00:00:00+00:00,2018-02-02 17:54:00,2018-02-02 19:51:00,2018,44.0


In [44]:
print("Idle times:\n",temp.loc[7:8].idle_time_minutes,"\n")
print("Start times:\n",temp.loc[7:8].start_Datetime,"\n")
print("Charging times:\n",temp.loc[7:8].Charging_Time__minutes_)

Idle times:
 7   -142.0
8     44.0
Name: idle_time_minutes, dtype: float64 

Start times:
 7   2018-02-02 17:41:00
8   2018-02-02 17:54:00
Name: start_Datetime, dtype: datetime64[ns] 

Charging times:
 7    155.0
8    117.0
Name: Charging_Time__minutes_, dtype: float64


The example above shows that something is wrong. The first observations starts at 17:41 and has a charging time of 155 minutes. It is therefor not idle before 20:16, but the next car starts charging at 17:54.

After doubble checking the data (link in the example above) it appears that it is due to some technical errors from the way the data is recorded. The observations are therefore removed.

In [45]:
#df = df[df['idle_time_minutes'] >= 0]
#df.shape

## Coordinates
The dataset does not provide us with excat coordinates for each station name but instead with an adress. Therefore [google maps](https://www.google.com/maps/) is used to look up each adresse and retrieve the coordinates. 

In [46]:
for name in names:
    temp = df[df['Station_Name'] == name]
    print(name)
    print("Address:", temp.Station_Address.unique())
    print("City:", temp.City.unique())
    print("State:", temp.State_Province.unique())
    print("Postal_code", temp.Postal_Code.unique(),"\n")

COMM VITALITY / 1104 SPRUCE1
Address: ['1100 Spruce St']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

COMM VITALITY / 1000WALNUT1
Address: ['900 Walnut St']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

BOULDER / REC CENTER ST2
Address: ['1305-1335 Gillaspie Dr' '1360 Gillaspie Dr']
City: ['Boulder']
State: ['Colorado']
Postal_code [80305] 

BOULDER / BASELINE ST1
Address: ['900 Baseline Rd' '600 Baseline Rd']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

BOULDER / ATRIUM ST1
Address: ['1770 13th St']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

BOULDER / ALPINE ST1
Address: ['1275 Alpine Ave']
City: ['Boulder']
State: ['Colorado']
Postal_code [80304] 

COMM VITALITY / 1000WALNUT2
Address: ['900 Walnut St' '949-959 Walnut St']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

COMM VITALITY / 1400 WALNUT1
Address: ['1400 Walnut St']
City: ['Boulder']
State: ['Colorado']
Postal_code [80302] 

BOULDER / FACILITIES ST1
Add

In [47]:
# the coordinates below are gathered from google maps
coordinate_dict = {'COMM VITALITY / 1104 SPRUCE1': [40.018481961484376, -105.28106600728856],
                  'COMM VITALITY / 1000WALNUT1': [40.01599429161985, -105.28247311646552],
                   'BOULDER / REC CENTER ST2': [39.97488130864729, -105.24921913913307],
                   'BOULDER / BASELINE ST1': [40.000148228895775, -105.2824371990558],
                   'BOULDER / ATRIUM ST1' : [40.015504096823335, -105.27723026116516],
                   'BOULDER / ALPINE ST1' : [40.02618558388287, -105.28078346640017],
                   'COMM VITALITY / 1000WALNUT2': [40.01606168623948, -105.28247453106862],
                   'COMM VITALITY / 1400 WALNUT1': [40.01707529163262, -105.27642310718852],
                   'BOULDER / FACILITIES ST1': [40.015368570009656, -105.27686715248305],
                   'BOULDER / REC CENTER ST1': [39.97473577002263, -105.2486322603292],
                   'COMM VITALITY / 1500PEARL1': [40.01850960088779+0.0001, -105.27575240968494],
                   'COMM VITALITY / 1500PEARL2': [40.01850960088779, -105.27575240968494],
                   'BOULDER / JUNCTION ST1': [40.02430279407053, -105.25178386118644],
                   'COMM VITALITY / BOULDER JCTN': [40.0260763826729, -105.2508954999583],
                   'COMM VITALITY / 1100WALNUT1': [40.01646322020759, -105.28058263149846],
                   'BOULDER / N BOULDER REC 1': [40.03280509665776, -105.28100769708102],
                   'BOULDER / BOULDER PARK S2': [40.01456428047208+0.0001, -105.27929534336563],
                   'BOULDER / BOULDER PARK S1': [40.01456428047208, -105.27929534336563],
                   'COMM VITALITY / 2200 BROADWAY1': [40.020085389964436, -105.27899882466563],
                   'BOULDER / EAST REC': [39.99223898595327, -105.22007446761407],
                   'BOULDER / ANNEX ST1': [40.02397978680573, -105.18270995242484],
                   'BOULDERJUNCTION / JUNCTION ST1': [40.025028, -105.250873],
                   'BOULDER / RESERVOIR ST1': [40.07206066731993+0.0001, -105.22884562487955],
                   'BOULDER / RESERVOIR ST2': [40.07206066731993, -105.22884562487955],
                   'BOULDER / CARPENTER PARK1': [40.01338145833486+0.0001, -105.25423010835927],
                   'BOULDER / CARPENTER PARK2': [40.01338145833486, -105.25423010835927],
                   'BOULDER / AIRPORT ST1': [40.03736647327974, -105.23060803081994]
                  }

In [48]:
# Construct vector with coordinates for every observation
longitudes = []
latitudes = []
for i in range(0,len(df)):
    latitudes.append(coordinate_dict[df.iloc[i].Station_Name][0])
    longitudes.append(coordinate_dict[df.iloc[i].Station_Name][1])

# add to dataframe
df['latitude'] = latitudes 
df['longitude'] = longitudes

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


## Add weekday
For both analysis and modelling the specific weekday of an event can be describing. The column `weekday` is therefore added with the relevant information. The start time of the event is used as the `weekday`. Accordingly to the column a week starts with 0 = monday and ends with sunday = 6.

In [49]:
df['weekday'] = df.start_Datetime.dt.dayofweek

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
def dayNameFromWeekday(weekday):
    if weekday == 0:
        return "Monday"
    if weekday == 1:
        return "Tuesday"
    if weekday == 2:
        return "Wednesday"
    if weekday == 3:
        return "Thursday"
    if weekday == 4:
        return "Friday"
    if weekday == 5:
        return "Saturday"
    if weekday == 6:
        return "Sunday"

In [51]:
df['weekday_name'] = df['weekday'].apply(dayNameFromWeekday)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
df.head()

,Station_Name,Station_Address,City,State_Province,Postal_Code,Transaction_Date,Transaction_Start_Time,Charging_Time__minutes_,Energy__kWh_,GHG_Savings__kg_,...,start_Time_Datetime,Transaction_Date_Datetime,start_Datetime,end_Datetime,year,idle_time_minutes,latitude,longitude,weekday,weekday_name
0,COMM VITALITY / 1104 SPRUCE1,1100 Spruce St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:43:00+00,128.0,2.84,1.19,...,2021-01-21 17:43:00+00:00,2018-01-31 00:00:00+00:00,2018-01-31 17:43:00,2018-01-31 19:51:00,2018,929.0,40.018482,-105.281066,2,Wednesday
1,COMM VITALITY / 1000WALNUT1,900 Walnut St,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 17:16:00+00,68.0,5.66,2.38,...,2021-01-21 17:16:00+00:00,2018-01-31 00:00:00+00:00,2018-01-31 17:16:00,2018-01-31 18:24:00,2018,228.0,40.015994,-105.282473,2,Wednesday
2,BOULDER / REC CENTER ST2,1305-1335 Gillaspie Dr,Boulder,Colorado,80305,2018/01/31 00:00:00+00,2021/01/21 13:52:00+00,73.0,3.94,1.65,...,2021-01-21 13:52:00+00:00,2018-01-31 00:00:00+00:00,2018-01-31 13:52:00,2018-01-31 15:05:00,2018,2356.0,39.974881,-105.249219,2,Wednesday
3,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 13:54:00+00,40.0,4.42,1.86,...,2021-01-21 13:54:00+00:00,2018-01-31 00:00:00+00:00,2018-01-31 13:54:00,2018-01-31 14:34:00,2018,1259.0,40.000148,-105.282437,2,Wednesday
4,BOULDER / BASELINE ST1,900 Baseline Rd,Boulder,Colorado,80302,2018/01/31 00:00:00+00,2021/01/21 11:51:00+00,42.0,4.60,1.93,...,2021-01-21 11:51:00+00:00,2018-01-31 00:00:00+00:00,2018-01-31 11:51:00,2018-01-31 12:33:00,2018,81.0,40.000148,-105.282437,2,Wednesday


### Save dataframe

In [56]:
#df.to_csv("../data/Electric_Vehicle_Charging_Station_Energy_Consumption_cleaned.csv",index=False)

Use pickle as this saves the dataframe with the datetime formats.

In [57]:
#df.to_pickle("../data/Electric_Vehicle_Charging_Station_Energy_Consumption_cleaned.pkl")

In [58]:
df.shape

(16023, 24)